# CGI basic contrast tool
### Goal

This notebook goal has helped to understand step by step how to get mean contrast Enid Dark Hole (DH) for the CoronaGraphic Instrument (CGI) of the Roman Space Telescope (RST) with WebbPSF.
To do it, we must understand how get a PSF in the final focal plane. Then how get a direct PSF to normalize doses PSF in raw contrast.


### Installation and before use it

Befaore start this notebook, check your WebbPSF installation and your git Head must be on this [fork](https://github.com/davidbourgeois/webbpsf). Please follow [installation instructions here](https://webbpsf.readthedocs.io/en/latest/installation.html#installing-with-conda-but-not-astroconda). 

First, I suggest to use [roman_cgi_demo notebook](https://github.com/davidbourgeois/webbpsf/blob/develop/notebooks/roman_cgi_demo.ipynb) from WebbPSF.


## PSF
In this section we will display our first CGI PSF in WebbPSF.

In [ ]:
#WebbPSF requierement
%pylab inline --no-import-all
from matplotlib.colors import LogNorm
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'

import os
os.environ['WEBBPSF_PATH'] = '<your_path>/webbpsf_data'
import webbpsf
from webbpsf import roman
import ipywidgets
from astropy.io import fits

In [ ]:
import logging

#Get WebbPSF logs
webbpsf.setup_logging()

#Remove useless logs warning
log = logging.getLogger()
mplfm_logger = logging.getLogger('matplotlib.font_manager')
mplcb_logger = logging.getLogger('matplotlib.colorbar')
mplt_logger = logging.getLogger('matplotlib.ticker')
mplbe_logger = logging.getLogger('matplotlib.backends')

mplfm_logger.setLevel(logging.WARNING)
mplcb_logger.setLevel(logging.WARNING)
mplt_logger.setLevel(logging.WARNING)
mplbe_logger.setLevel(logging.WARNING)

In [ ]:
#Create a coronagraph instancy
SPC = roman.CGI(mode='CHARSPC_F770')

#Create and display optical path of an unaberrated coronographic PSF
psf = SPC.calc_psf(nlambda=1, fov_arcsec=1.6, display=True)

In [ ]:
#Display only PSF on last focal plan in scientific camera
psf_fit = psf[0].data #Extract data from generated fit by calc_psf()

plt.imshow(psf_fit, norm = LogNorm(vmin=1e-13))
plt.colorbar()

## Direct PSF & DH
In this section, we will use the 2 main contrast functions.

In [ ]:
#Raw_coronagraph remove FPM to create a direct PSF
direct = SPC.raw_coronagraph()

#Create and display optical path of direct PSF
psf_direct = direct.calc_psf(nlambda=1, fov_arcsec=1.6, display=True)

In [ ]:
#Display only direct PSF on last focal plan in scientific camera
psf_direct_fit = psf_direct[0].data #Extract data from generated fit by calc_psf()

plt.imshow(psf_direct_fit, norm = LogNorm(vmin=1e-9))
plt.colorbar()

In [ ]:
#We definite norm with maximun of direct PSF
norm = np.max(psf_direct_fit)

#We normalize our PSF in contrast
contrast = psf_fit/ norm

#First display in contrast
plt.imshow(contrast, norm = LogNorm(vmin=1e-9))
plt.colorbar()

In [ ]:
#Next step we definite create a DH wor this coronagraph
iwa = 3 #inner working area
owa = 9 #outer working area 

dh_mask = SPC.working_area(im=contrast, inner_rad=iwa, outer_rad=owa) #method to create DH
dh_mask = SPC.WA #DH attribut

#Display our DH
plt.imshow(dh_mask)
plt.colorbar()

## Raw contrast
In this section, we will get mean contrast and use raw_contrast function for another coronagraph.

In [ ]:
# We multiply our contrast figure with our his associate DH
contrast_dh = contrast*dh_mask

plt.imshow(contrast_dh)
plt.colorbar()

In [ ]:
#Finaly get mean contrast in this DH
mean_contrast = np.mean(contrast_dh[np.where(contrast_dh != 0)])
print(mean_contrast)

In [ ]:
# Now we can do same for all configuration in only 2 lines !

DISKSPC = roman.CGI(mode='DISKSPC_F721')

DISKSPC.raw_contrast(display=True)